In [1]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
#from process_craigslist import Process
from scrape_craigslist import Scrape
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas_profiling
from scipy import stats
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import pickle
import glob   
import os
import io
import re

In [90]:
li=[]
for filename in glob.glob(r"C:\Users\Silmi\Desktop\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)

In [82]:
makemodel=[]
make_dict=dict(df['Make&Model'].value_counts())
for i,x in make_dict.items():
    if(x>80):
        makemodel.append(i)
    

In [93]:
df_spelled=Spell(frame)
df_replaced=Replace(df_spelled)
for car in makemodel:
    try:
        
        proccessed=Process(df_replaced,car)
        df=Outliers(proccessed)
        X=df.drop(columns='price:')
        y=df['price:']
        train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)
        rf = RandomForestRegressor( random_state = 42)
        rf.fit(train_features, train_labels);
        # Calculate mean absolute percentage error (MAPE)
        y_pred=rf.predict(test_features)
        errors=np.abs((test_labels - y_pred))
        mape = 100 * (errors / test_labels)
        accuracy = 100 - np.mean(mape)
        print(f'{car} Accuracy:', round(accuracy, 2), '%.')
    except:
        continue

toyota prius Accuracy: 96.81 %.
honda civic Accuracy: 96.58 %.
toyota camry Accuracy: 97.19 %.
honda accord Accuracy: 95.85 %.
toyota tacoma Accuracy: 52.26 %.
chevrolet silverado Accuracy: 94.79 %.
toyota corolla Accuracy: 63.32 %.
ford f150 Accuracy: 95.07 %.
dodge ram Accuracy: 94.13 %.
toyota 4runner Accuracy: 96.99 %.
nissan altima Accuracy: 96.73 %.
jeep wrangler Accuracy: 39.92 %.
bmw 328i Accuracy: 96.5 %.
honda crv Accuracy: 56.87 %.
ford mustang Accuracy: 94.85 %.
honda odyssey Accuracy: 96.11 %.
toyota sienna Accuracy: 96.19 %.
volkswagen jetta Accuracy: 95.92 %.
ford f250 Accuracy: 93.07 %.
mini cooper Accuracy: 95.79 %.
ford focus Accuracy: 95.59 %.
gmc sierra Accuracy: 94.69 %.
toyota tundra Accuracy: 96.97 %.
nissan sentra Accuracy: 97.26 %.
ford fusion Accuracy: 73.75 %.
ford explorer Accuracy: 95.94 %.
chevrolet tahoe Accuracy: 96.6 %.
ford ranger Accuracy: 96.28 %.
hyundai sonata Accuracy: 96.14 %.
audi a4 Accuracy: 96.73 %.
ford f350 Accuracy: 94.32 %.
hyundai elantr

In [819]:
link=['']
mape=np.mean(np.abs((test_labels - y_pred) / test_labels)) * 100

In [ ]:
raw=Scrape(link)
df1=Spell(raw)
df2=Replace(df1)
df3=Process(df2,"")
x=df.drop('price:',axis=1)
y=df['price:']
print(2**rf.predict(x))
print(y)

In [766]:
#profile=_df.profile_report()

In [18]:
def Process(df,makemodel=0):

    df= df[df['price:'].notna()]
    df['Make&Model'] = df[['make:', 'model:']].apply(lambda x: ' '.join(x), axis=1)
    if(makemodel!=0):
        df=df[df['Make&Model']==makemodel]
        df=df[['year:','condition:','odometer:',
               'cylinders:','status:','price:']]
    else:
        df=df[['Make&Model','year:','condition:','odometer:',
               'cylinders:','status:','price:']]
    df=df.drop_duplicates()
    df=df.reset_index(drop=True)
    return df

In [7]:
def Spell(df_):
       
    df_= df_[df_['price:'].notna()]
    df_['make:'].replace(['mazda','MAZDA'],'mazda',inplace=True)
    df_['make:'].replace(['MITSUBISHI'],'mitsubishi',inplace=True)
    df_['make:'].replace(['mercedes','Mercedes-benz','Mercedez','Mersedes','mbz','Mbz',
                          'mercedez','mercedes-benz','Mercedes',
                          'Benz','MERCEDES','Merceds','MERCEDES-BENZ',
                          'Mercedes-Benz','mersedes'],'mercedes-benz',inplace=True)
    df_['make:'].replace([ 'Cevrolet','Chaey','Chebrolet','Chevelle','Chevorlet','Chevroler',
                          'Chevrolet','Chevrolete','Chevy.','chevrolet','Chverolet','Shevrolet',
                          'chevrolet','Chevi','CHEVY','Chevrolet',
                          'chevorlet','chevy','CHEVRLET','Cheverolet','Chevy','Chev',
                          'CHEVROLET','chev'],'chevrolet',inplace=True)
    df_['make:'].replace(['volkseagen','volkswagon','volkwagen', 'Volkswagon',
                          'Volkswangen','Volkwagen','volkswagen',
                          'Vw','VW','vw','VOLKSWAGEN','V.W'],'volkswagen',inplace=True)
    df_['make:'].replace(['Toyota','TOYOYA','toyta','Toyot','Toyoda','TOYOTA',
                          'toyota'],'toyota',inplace=True)
    df_['make:'].replace(['Mini','MINI'],'mini',inplace=True)
    df_['make:'].replace(['Isuzu','Izusu','Izuzu','izuzu','isuzu'],'isuzu',inplace=True)
    df_['make:'].replace(['huyndai','hyndai','hynduia','Hyuandi','Hunday','hundai','hyundia',
                          'HYUNDAI'],'hyundai',inplace=True)
    df_['make:'].replace(['Bmw','bimmer','Bmw'],'bmw',inplace=True)
    df_['make:'].replace(['Audi','AUDI'],'audi',inplace=True)
    df_['make:'].replace(['LEXUS','Lexus'],'lexus',inplace=True)
    df_['make:'].replace(['KIA','Kia'],'kia',inplace=True)
    df_['make:'].replace(['Ford','FORD','for'],'ford',inplace=True)
    df_['make:'].replace(['HUMMER','Hummer','hummer','Gmc','GMC','Gmc'],'gmc',inplace=True)
    df_['make:'].replace(['HONDA','Houda','honda'],'Honda',inplace=True)
    df_['make:'].replace(['jeep','Jeeb','JEEP'],'Jeep',inplace=True)
    df_['make:'].replace(['audi','AUDI'],'Audi',inplace=True)
    df_['make:'].replace(['VOLVO', 'volvo'],'Volvo',inplace=True)
    df_['make:'].replace(['Infinti','infinti','INFINITI','infiniti',
                          'infinity'],'Infinity',inplace=True)
    df_['make:'].replace(['FIAT', 'fiat'],'Fiat',inplace=True)
    df_['make:'].replace(['Nissin','NISSAN','Nissan'],'nissan',inplace=True)
    df_['make:'].replace(['PORSCHE','porsche'],'Porsche',inplace=True)
    df_['make:'].replace(['model','Model','TESLA','TELSA','Tesla'],'tesla',inplace=True)
    df_['make:'].replace(['alfa Romeo','Alfa','alfa'],'Alfa Romeo',inplace=True)
    df_['make:'].replace(['land-rover','LANDROVER','Rover','rover','land','Land rover',
                          'land rover'],'land rover',inplace=True)
    df_['make:'].replace([ 'chryler','chrystler','chyrsler','chysler',
                          'chrysler','CRYSLER','Cheysler'],'chrysler',inplace=True)
    df_['make:'].replace(['pontaic','pontiac'],'pontiac',inplace=True)

    df_['make:']=df_['make:'].apply(lambda x: x.lower())
    df_['model:']=df_['model:'].apply(lambda x: x.lower())
    df_['trim:'].fillna('', inplace=True)
    
    df_['trim:']=df_['trim:'].apply(lambda x: x.lower() if (type(x)=='str') else x)
    df_['model:']=df_['model:'].apply(lambda x: x.replace('-',""))
    df_['model:']=df_['model:'].apply(lambda x: x.replace(',',""))
    df_['model:']=df_['model:'].apply(lambda x: x.replace('.',""))
    df_['model:'] = np.where(df_['model:'] == 'benz', df_['trim:'], df_['model:'])
    
   
    
    return df_

In [8]:
def Replace(df):
    df= df[df['price:'].notna()]
    df['year:']=df['year:'].astype(int)
    df['year:']=df['year:'].apply(lambda x:(2022-x))
    df['price:']=df['price:'].astype(int)
    df=df.drop_duplicates()
    col_drop=['trim:','color:','fuel:','type:','drive:','transmission:']
    df.drop(columns=col_drop, inplace=True)
    df=df.reset_index(drop=True)
    
    df['condition:']=df['condition:'].fillna("good")
    df['cylinders:']=df['cylinders:'].fillna("4")
    df['cylinders:']=df['cylinders:'].replace('other','4')
    df['cylinders:']=df['cylinders:'].astype(int)
    
    cleanup_nums ={
    "condition:": {"new":2,"good": 1, "excellent": 2,"like": 2,"fair": 0,"salvage": 0},
     "transmission:": {"automatic": 1, "manual": 2, "other":3 },
    "status:":{"clean": 1, "salvage": 0, "missing":0,"lien":0,"parts":0,"rebuilt":0},
    "drive:":{"4wd": 3, "rwd": 2, "fwd":1},
    "fuel:":{'electric':5,"gas": 4, "diesel": 3, "hybrid":2,"other":1}}
    
    df= df.replace(cleanup_nums)
    
    return df

    

In [9]:
def Outliers(df): 
    #handling outliers 
    df=df[(np.abs(stats.zscore(df['year:'])) <2.5)]
    df=df[(np.abs(stats.zscore(df['odometer:'])) <3)]
    df=df[(np.abs(stats.zscore(df['price:'])) <3)]
    df=df[df['price:']>1000]
    
    #handling price skewness
    if(df['price:'].skew()>1 or df['price:'].skew()<-1):
        df['price:']=np.log2(df['price:'])
    cyl_min=df['cylinders:'].value_counts().min()
    cyl_max=df['cylinders:'].value_counts().max()
    if((cyl_min/cyl_max)<(0.2)or(cyl_min/cyl_max==1)):
        df.drop(columns='cylinders:',axis=1,inplace=True)
    return df